In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
import joblib

# Load dataset
df = pd.read_csv("pg_coliving_dataset.csv")

# Combine rare shearing categories
df['shearing'] = df['shearing'].replace({'four': 'shared', 'multi': 'shared'})

# Add interaction feature: meals + shearing
df['meals_shearing'] = df['meals_avilable'] + '_' + df['shearing']

# Select relevant features
features = [
    'city', 'area', 'location', 'zone', 'shearing', 'best_suit_for',
    'meals_avilable', 'notic_period', 'lock_in_period', 'amenities_count',
    'in_time_at_night', 'non_veg_allowed', 'opposite_gender_allowed',
    'visitors_allowed', 'gurdian_allowed', 'drinking_allowed', 'smoking_allowed',
    'meals_shearing'
]
target = 'rent_price'

# Filter and clean
df = df[features + [target]].dropna()
df['amenities_count'] = pd.to_numeric(df['amenities_count'], errors='coerce').fillna(0)
df['rent_price'] = pd.to_numeric(df['rent_price'], errors='coerce')
df.dropna(inplace=True)

# Features and label
X = df.drop(columns='rent_price')
y = df['rent_price']

# Preprocessing
cat_cols = X.select_dtypes(include='object').columns.tolist()
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
], remainder='passthrough')

# Model pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train/test split and training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

# Save model
joblib.dump(pipeline, "pg_rent_final_model.pkl")
print("✅ Model trained and saved as pg_rent_final_model.pkl")


✅ Model trained and saved as pg_rent_final_model.pkl


In [2]:
import pandas as pd
import joblib

# Load the saved model
model = joblib.load("pg_rent_final_model.pkl")

def predict_pg_rent_refined(
    city, area, location, zone, shearing, best_suit_for,
    meals_avilable, notic_period, lock_in_period, amenities_count,
    in_time_at_night, non_veg_allowed, opposite_gender_allowed,
    visitors_allowed, gurdian_allowed, drinking_allowed, smoking_allowed
):
    # Combine shearing groups
    shearing = shearing if shearing not in ['four', 'multi'] else 'shared'
    meals_shearing = meals_avilable + "_" + shearing

    input_data = pd.DataFrame([{
        'city': city,
        'area': area,
        'location': location,
        'zone': zone,
        'shearing': shearing,
        'best_suit_for': best_suit_for,
        'meals_avilable': meals_avilable,
        'notic_period': notic_period,
        'lock_in_period': lock_in_period,
        'amenities_count': amenities_count,
        'in_time_at_night': in_time_at_night,
        'non_veg_allowed': non_veg_allowed,
        'opposite_gender_allowed': opposite_gender_allowed,
        'visitors_allowed': visitors_allowed,
        'gurdian_allowed': gurdian_allowed,
        'drinking_allowed': drinking_allowed,
        'smoking_allowed': smoking_allowed,
        'meals_shearing': meals_shearing
    }])

    prediction = model.predict(input_data)[0]
    return round(prediction, 2)

# Example usage
predicted_rent = predict_pg_rent_refined(
    city='nagpur',
    area='manewada',
    location='manewada, nagpur',
    zone='south',
    shearing='private',
    best_suit_for='students',
    meals_avilable='no',
    notic_period='15 days',
    lock_in_period='30 days',
    amenities_count=3,
    in_time_at_night='10 PM',
    non_veg_allowed='yes',
    opposite_gender_allowed='no',
    visitors_allowed='yes',
    gurdian_allowed='yes',
    drinking_allowed='no',
    smoking_allowed='no'
)

print(f"✅ Predicted Rent: ₹{predicted_rent}")


✅ Predicted Rent: ₹5825.0


In [3]:
predicted_rent = predict_pg_rent_refined(
    city='nagpur',
    area='jaitala',
    location='jaitala, nagpur',
    zone='south',
    shearing='double',
    best_suit_for='students',
    meals_avilable='no',
    notic_period='15 days',
    lock_in_period='30 days',
    amenities_count=3,
    in_time_at_night='11 PM',
    non_veg_allowed='yes',
    opposite_gender_allowed='no',
    visitors_allowed='yes',
    gurdian_allowed='yes',
    drinking_allowed='no',
    smoking_allowed='no'
)

print(f"✅ Predicted Rent: ₹{predicted_rent}")

✅ Predicted Rent: ₹3805.0


In [4]:
predicted_rent = predict_pg_rent_refined(
    city='nagpur',
    area='omkar nagar',
    location='omkar nagar, nagpur',
    zone='south',
    shearing='double',
    best_suit_for='students',
    meals_avilable='yes',
    notic_period='15 days',
    lock_in_period='30 days',
    amenities_count=3,
    in_time_at_night='11 PM',
    non_veg_allowed='yes',
    opposite_gender_allowed='no',
    visitors_allowed='yes',
    gurdian_allowed='yes',
    drinking_allowed='no',
    smoking_allowed='no'
)

print(f"✅ Predicted Rent: ₹{predicted_rent}")

✅ Predicted Rent: ₹6985.0
